# 01 · Data Cleaning & Preparation (MIMIC‑III)
Project: **Early detection of sepsis in ICU patients**

This notebook builds the initial cohort and **sepsis label**, then previews a small, clean analysis table for Assignment 2.

**Tables expected in `data/`:** `PATIENTS.csv`, `ADMISSIONS.csv`, `ICUSTAYS.csv`, `DIAGNOSES_ICD.csv`, `D_ICD_DIAGNOSES.csv` (optional for lookup).


In [9]:

import os, re, pandas as pd, numpy as np
from datetime import timedelta
DATA_DIR = os.path.join('..', 'data')


In [10]:

# Load core tables
patients = pd.read_csv(os.path.join(DATA_DIR, 'PATIENTS.csv'),
                       usecols=['SUBJECT_ID','GENDER','DOB','DOD'],
                       parse_dates=['DOB','DOD'])
admissions = pd.read_csv(os.path.join(DATA_DIR, 'ADMISSIONS.csv'),
                         usecols=['SUBJECT_ID','HADM_ID','ADMITTIME','DISCHTIME','DEATHTIME'],
                         parse_dates=['ADMITTIME','DISCHTIME','DEATHTIME'])
icustays = pd.read_csv(os.path.join(DATA_DIR, 'ICUSTAYS.csv'),
                       usecols=['ROW_ID','SUBJECT_ID','HADM_ID','ICUSTAY_ID','INTIME','OUTTIME','LOS'],
                       parse_dates=['INTIME','OUTTIME'])
display(patients.head(), admissions.head(), icustays.head())


ValueError: Usecols do not match columns, columns expected but not found: ['SUBJECT_ID', 'DOB', 'GENDER', 'DOD']

In [ ]:

# Compute age at ICU admission and basic cohort
icu = icustays.merge(admissions[['SUBJECT_ID','HADM_ID','ADMITTIME']], on=['SUBJECT_ID','HADM_ID'], how='left')
icu = icu.merge(patients[['SUBJECT_ID','GENDER','DOB']], on='SUBJECT_ID', how='left')
icu['AGE_YEARS'] = (icu['ADMITTIME'] - icu['DOB']).dt.days / 365.25
icu['AGE_YEARS'] = icu['AGE_YEARS'].clip(lower=0, upper=120)
cohort = icu[['SUBJECT_ID','HADM_ID','ICUSTAY_ID','INTIME','OUTTIME','LOS','GENDER','AGE_YEARS']].copy()
cohort.head()


NameError: name 'icustays' is not defined

In [ ]:

# Build sepsis label from ICD-9 codes in DIAGNOSES_ICD.csv
diag = pd.read_csv(os.path.join(DATA_DIR, 'DIAGNOSES_ICD.csv'),
                   usecols=['SUBJECT_ID','HADM_ID','ICD9_CODE'])
def normalize_icd(code):
    import re, pandas as pd
    if pd.isna(code): return None
    return re.sub(r'\D', '', str(code))
diag['ICD9_NORM'] = diag['ICD9_CODE'].apply(normalize_icd)
sepsis_codes = {'99591','99592','78552'}
hadm_sepsis = (diag['ICD9_NORM'].isin(sepsis_codes)).groupby([diag['SUBJECT_ID'], diag['HADM_ID']]).any().reset_index()
hadm_sepsis.rename(columns={0:'SEPSIS'}, inplace=True)
cohort = cohort.merge(hadm_sepsis, on=['SUBJECT_ID','HADM_ID'], how='left')
cohort['SEPSIS'] = cohort['SEPSIS'].fillna(False).astype(int)
cohort.head(10)


ValueError: Usecols do not match columns, columns expected but not found: ['SUBJECT_ID', 'ICD9_CODE', 'HADM_ID']

### Save a small preview for sanity-check

In [ ]:

out = os.path.join('..','data','analysis_table_preview.csv')
cohort.head(20).to_csv(out, index=False)
out


NameError: name 'cohort' is not defined

Next: use `scripts/make_analysis_dataset.py` to add vitals/labs and produce plots for slides.